In [1]:
import os

In [2]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Rupesh-132"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d0abd907d6e7b5b15f194e16de7c132a9ef9cdef"

Entity Definition for model evaluation

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


Configuration Manager for model evaluation strategy

In [7]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection.mlflow"
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from Credit_Card_Fraud_Detection.utils.common import save_json

In [10]:
class ModelEvaluationStrategy:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        #classification_report = classification_report(actual,pred)
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual,pred)
        f1 = f1_score(actual,pred)
        matthews_corr = matthews_corrcoef(actual,pred)
        
        return accuracy,precision,recall,f1,matthews_corr



    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (accuracy,precision,recall,f1,matthews_corr) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "precision": precision, "recall_score": recall,"f1_score":f1,"matthews_corr":matthews_corr}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall_score", recall)
            mlflow.log_metric("f1_score",f1)
            mlflow.log_metric("matthews_corrcoef",matthews_corr)
            

        
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")
    
     


    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluationStrategy(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-05 10:05:56,997: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-05 10:05:56,999: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-05 10:05:57,000: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-05 10:05:57,000: INFO: common: created directory at: artifacts]
[2024-01-05 10:05:57,000: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-05 10:05:58,445: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/01/05 10:06:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 2
Created version '2' of model 'RandomForestClassifier'.
